In [595]:
import os
import csv
import names
import random
import sqlite3
import pandas as pd
import numpy as np
from faker import Faker

faker = Faker('en_GB')

In [596]:
if os.path.exists('./mydb.db'):
    os.remove('./mydb.db')
conn = sqlite3.connect('./mydb.db')
c = conn.cursor()

# Defining the Tables

In [597]:
c.execute("""
            CREATE TABLE IF NOT EXISTS Players
            ([shirt_num] INTEGER PRIMARY KEY,
            [first_name] TEXT,
            [last_name] TEXT,
            [role] TEXT,
            [tenure] INTEGER,
            [salary] FLOAT,
            [address] TEXT,
            [matches_played] INT,
            [goals_scored] INT,
            [assists] INT
            )
            """)

In [598]:
c.execute("""
            CREATE TABLE IF NOT EXISTS Matches
            ([id] INTEGER PRIMARY KEY,
            [competition_name] TEXT KEY,
            [rival_team] TEXT,
            [date] DATE,
            [location] TEXT,
            [points] INT,
            [line_up] TEXT
            )
            """)

In [599]:
c.execute("""
            CREATE TABLE IF NOT EXISTS CoachingStaff
            ([employee_id] INTEGER PRIMARY KEY,
            [first_name] TEXT,
            [last_name] TEXT,
            [role] TEXT,
            [tenure] INTEGER,
            [salary] FLOAT,
            [address] TEXT
            )
            """)

In [600]:
c.execute("""
            CREATE TABLE IF NOT EXISTS LFCMembers
            ([supporter_id] INTEGER PRIMARY KEY,
            [first_name] TEXT,
            [last_name] TEXT,
            [age] INTEGER,
            [address] TEXT
            )
            """)

In [601]:
c.execute("""
            CREATE TABLE IF NOT EXISTS Tickets
            ([ticket_id] INTEGER PRIMARY KEY,
            [match_id] INTEGER KEY,
            [seat_number] INTEGER KEY,
            [supporter_id] INTEGER KEY,
            [competition_name] TEXT KEY,
            [cost] INT
            )
            """)

In [602]:
c.execute("""
            CREATE TABLE IF NOT EXISTS Competitions
            ([competition_name] TEXT PRIMARY KEY,
            [prize] INTEGER,
            [status] TEXT
            )
            """)

In [603]:
c.execute("""
            CREATE TABLE IF NOT EXISTS Seats
            ([seat_number] INTEGER PRIMARY KEY,
            [stand] TEXT,
            [block] TEXT,
            [home_supporters] BOOL
            )
            """)

In [604]:
c.execute("""
            CREATE TABLE IF NOT EXISTS LFCShop
            ([item_id] INTEGER PRIMARY KEY,
            [supporter_id] INTEGER KEY,
            [amount] INTEGER,
            [price] INTEGER
            )
            """)

# Creating the Database

#### Coaching Staff

In [605]:
coaches_first_name = ['Jurgen']
coaches_last_name = ['Klopp']

for i in range(9):
    coaches_first_name.append(names.get_first_name(gender='male'))
    coaches_last_name.append(names.get_last_name())

emp_id = list(set([faker.pyint() for _ in range(100)]))[:10]
roles = ['GeneralManager'] + random.choices(['GKCoach', 'RunningCoach', 'ShootingCoach', 'PassingCoach', 'StategyCoach'], k=9)
tenure = [random.randint(1, 7) for _ in range(10)]
salary = [random.randint(2000, 10000) for _ in range(10)]
address = [faker.street_address() for _ in range(10)]
columns = ['employee_id', 'first_name', 'last_name', 'role', 'tenure', 'salary', 'address']

coaches = pd.DataFrame([emp_id, coaches_first_name, coaches_last_name, roles, tenure, salary, address]).T
coaches.columns = columns
coaches.to_csv('csvs/coaches.csv', index=False)

with open('csvs/coaches.csv', 'r') as data:
    coaches_dict = csv.DictReader(data)
    db_format = [(i['employee_id'], i['first_name'], i['last_name'], i['role'], i['tenure'], i['salary'], i['address']) for i in coaches_dict]

c.executemany("INSERT INTO CoachingStaff (employee_id, first_name, last_name, role, tenure, salary, address) \
    VALUES (?, ?, ?, ?, ?, ?, ?)", db_format)

#### Players

In [606]:
shirt_number = np.arange(1, 16).tolist()
first_name = ['Alisson', 'Joel', 'Joe', 'Ibrahima', 'Virgil', 'Trent', 'Andrew', 'Jordan', 'James', 'Thiago', 'Diogo', 'Roberto',\
                'Mohamed', 'Sadio', 'Luis']
last_name = ['Becker', 'Matip', 'Gomez', 'Konate', 'Van-Dijk', 'Alexander-Arnold', 'Robertson', 'Henderson', 'Milner', 'Alcantara',\
                'Jota', 'Firmino', 'Salah', 'Mane', 'Diaz']
roles = ['GK'] + 6 * ['def'] + 4 * ['mid'] + 4 * ['fwd']
tenure = random.choices(np.arange(1, 8).tolist(), k=15)
salary = [float(i) for i in random.choices(np.arange(100, 300).tolist(), k=15)]
address = [faker.street_address() for _ in range(15)]
matches_played = random.choices(np.arange(10, 200).tolist(), k=15)
goals_scored = random.choices(np.arange(0, 40).tolist(), k=15)
assists = random.choices(np.arange(0, 40).tolist(), k=15)

players = pd.DataFrame([shirt_number, first_name, last_name, roles, tenure, salary, address, matches_played, goals_scored, assists]).T
players.columns = ['shirt_number', 'first_name', 'last_name' ,'roles', 'tenure', 'salary', 'address', 'matches_played', 'goals_scored', 'assists']
players.to_csv('csvs/players.csv', index=False)

with open('csvs/players.csv', 'r') as data:
    players_dict = csv.DictReader(data)
    db_format = [(i['shirt_number'], i['first_name'], i['last_name'], i['roles'], i['tenure'], i['salary'],\
        i['address'], i['matches_played'], i['goals_scored'], i['assists']) for i in players_dict]

c.executemany("INSERT INTO Players (shirt_num, first_name, last_name, role, tenure, salary, address, matches_played, goals_scored, assists)\
     VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", db_format)

#### Matches

In [607]:
pl_data = pd.read_csv('csvs/2019-20.csv')
teams_list = np.unique(pl_data['HomeTeam']).tolist()
teams_list.remove('Liverpool')

match_id = list(set([faker.pyint() for _ in range(100)]))[:19]
competitions = ['Champions_League', 'Premier_League', 'FA_Cup', 'Emirates_Cup', 'Carbao_cap']
rival_teams = teams_list.copy()
line_up = [random.choices(players['last_name'], k=11) for _ in range(19)]

dates = list(set(random.choices(pl_data['Date'], k=100)))[:19]
competition_names = random.choices(competitions, k=19)
locations = random.choices(['Anfield', 'Other'], k=19)
points = random.choices(['0', '1', '3'], k=19)


matches = pd.DataFrame([match_id, competition_names, rival_teams, dates, locations, points, line_up]).T
matches.columns = ['id', 'competition_name', 'rival_team', 'date', 'location', 'points', 'line_up']
matches.to_csv('csvs/matches.csv', index=False)


with open('csvs/matches.csv', 'r') as data:
    matches_dict = csv.DictReader(data)
    db_format = [(i['id'], i['competition_name'], i['rival_team'], i['date'], i['location'], i['points'], i['line_up']) for i in matches_dict]

c.executemany("INSERT INTO Matches (id, competition_name, rival_team, date, location, points, line_up) VALUES (?, ?, ?, ?, ?, ?, ?)", db_format)

#### Seats

In [608]:
seat_number = list(set(random.choices(np.arange(1, 100).tolist(), k=100)))[:50]
stand = random.choices(['low_tier', 'upper_tier', 'Kop', 'main_stand'], k=50)
block = random.choices(['A', 'B', 'C', 'D', 'E'], k=50)

seats = pd.DataFrame([seat_number, stand, block]).T
seats.columns = ['seat_number', 'stand', 'block']
seats['home_supporters'] = np.where(seats['stand'] == 'Kop', True, False)
seats.to_csv('csvs/seats.csv', index=False)

with open('csvs/seats.csv', 'r') as data:
    seats_dict = csv.DictReader(data)
    db_format = [(i['seat_number'], i['stand'], i['block'], i['home_supporters']) for i in seats_dict]

c.executemany("INSERT INTO Seats (seat_number, stand, block, home_supporters) VALUES (?, ?, ?, ?)", db_format)

#### Competitions

In [609]:
prize = [faker.pyint() for _ in range(len(competitions))]
status = random.choices(['relegated', 'qualified', 'won', 'lost'], k=len(competitions))

competitions_df = pd.DataFrame([competitions, prize, status]).T
competitions_df.columns = ['competition_name', 'prize', 'status']
competitions_df.to_csv('csvs/competitions.csv', index=False)

with open('csvs/competitions.csv', 'r') as data:
    comp_dict = csv.DictReader(data)
    db_format = [(i['competition_name'], i['prize'], i['status']) for i in comp_dict]

c.executemany("INSERT INTO Competitions (competition_name, prize, status) VALUES (?, ?, ?)", db_format)

#### LFC Members

In [610]:
supporter_id = list(set([faker.pyint() for _ in range(200)]))[:50]
first_names = [names.get_first_name('male') for _ in range(50)]
last_names = [names.get_last_name() for _ in range(50)]
age = random.choices(np.arange(20, 80).tolist(), k=50)
address = [faker.street_address() for _ in range(50)]

supporters = pd.DataFrame([supporter_id, first_names, last_names, age, address]).T
supporters.columns = ['supporter_id', 'first_name', 'last_name', 'age', 'address']
supporters.to_csv('csvs/lfc_supporters.csv', index=False)

with open('csvs/lfc_supporters.csv', 'r') as data:
    sup_dict = csv.DictReader(data)
    db_format = [(i['supporter_id'], i['first_name'], i['last_name'], i['age'], i['address']) for i in sup_dict]

c.executemany("INSERT INTO LFCMembers (supporter_id, first_name, last_name, age, address) VALUES (?, ?, ?, ?, ?)", db_format)

#### Tickets

In [611]:
tickets_id = list(set([faker.pyint() for _ in range(300)]))[:100]
match_id_tk = random.choices(match_id, k=100)
seat_number_tk = random.choices(seat_number, k=100)
supporter_id_tk = random.choices(supporter_id, k=100)
cost = random.choices(np.arange(100, 200).tolist(), k=100)

tickets = pd.DataFrame([tickets_id, match_id_tk, seat_number_tk, supporter_id_tk, cost]).T
tickets.columns = ['tickets_id', 'match_id', 'seat_number', 'supporter_id', 'cost']
tickets['competition_name'] = [matches[matches['id'] == id]['competition_name'].values[0] for id in tickets['match_id']]
tickets = tickets[['tickets_id', 'match_id', 'seat_number', 'supporter_id', 'competition_name', 'cost']]
tickets.to_csv('csvs/tickets.csv', index=False)

with open('csvs/tickets.csv', 'r') as data:
    tick_dict = csv.DictReader(data)
    db_format = [(i['tickets_id'], i['match_id'], i['seat_number'], i['supporter_id'], i['competition_name'], i['cost']) for i in tick_dict]

c.executemany("INSERT INTO Tickets (ticket_id, match_id, seat_number, supporter_id, competition_name, cost) VALUES (?, ?, ?, ?, ?, ?)", db_format)

#### LFC Shop

In [612]:
item_id = list(set([faker.pyint() for _ in range(200)]))[:50]
amount = random.choices(np.arange(0, 200).tolist(), k=50)
price = random.choices(np.arange(100, 5000).tolist(), k=50)
supporter_id_sh = random.choices(supporter_id[:10], k=50)

lfc_shop = pd.DataFrame([item_id, supporter_id_sh, amount, price]).T
lfc_shop.columns = ['item_id', 'supporter_id', 'amount', 'price']
lfc_shop.to_csv('csvs/lfc_shop.csv', index=False)


with open('csvs/lfc_shop.csv', 'r') as data:
    lfc_dict = csv.DictReader(data)
    db_format = [(i['item_id'], i['supporter_id'], i['amount'], i['price']) for i in lfc_dict]

c.executemany("INSERT INTO LFCShop (item_id, supporter_id, amount, price) VALUES (?, ?, ?, ?)", db_format)

# Interesting Insights

#### 1. The 5 top goal scorers in the team

In [613]:
c.execute("""
          SELECT first_name, last_name, role, goals_scored
          FROM Players
          ORDER BY goals_scored DESC
          LIMIT 5
          """)
pd.DataFrame(c.fetchall(), columns=['first_name', 'last_name', 'role', 'goals_scored'])

,first_name,last_name,role,goals_scored
0,Thiago,Alcantara,mid,35
1,Roberto,Firmino,fwd,33
2,Ibrahima,Konate,def,32
3,Sadio,Mane,fwd,31
4,Mohamed,Salah,fwd,28


#### 2. # of tickets sold in each competition

In [614]:
c.execute("""
          SELECT competition_name, COUNT(ticket_id)
          FROM Tickets
          GROUP BY competition_name
          """)

pd.DataFrame(c.fetchall(), columns=['competition_name', 'tickets_sold'])

,competition_name,tickets_sold
0,Carbao_cap,14
1,Champions_League,20
2,Emirates_Cup,41
3,FA_Cup,25


#### 3. Average salary of coaches

In [615]:
c.execute("""
          SELECT AVG(salary)
          FROM CoachingStaff
          """)
print(f"The average salary of the coaching staff is {c.fetchall()[0][0]}£")

The average salary of the coaching staff is 6499.4£


#### 4. Money spent by each supporter in the LFC Shop from the supporters who spent more than 5000£

In [616]:
c.execute("""
          SELECT s.supporter_id, sum(price) as money_spent
          FROM LFCMembers as l JOIN LFCShop as s on l.supporter_id = s.supporter_id
          GROUP BY s.supporter_id
          HAVING sum(price) > 5000
          """)
pd.DataFrame(c.fetchall(), columns=['supporter_id', 'money_spent'])

,supporter_id,money_spent
0,3608,13794
1,5634,8018
2,5639,10662
3,5650,19265
4,6152,11763
5,8192,7308
6,8707,28662
7,8723,14584


#### 5. First name of the players which their last name ends with 'son'

In [561]:
c.execute("""
          SELECT first_name
          FROM Players
          WHERE last_name LIKE '%son'
          """)

pd.DataFrame(c.fetchall(), columns=['first_name'])

,first_name
0,Andrew
1,Jordan


#### 6. Line-Ups for each day in the week of 10/08/2019 - 17/08/2019 and the team didn't lose

In [622]:
c.execute("""
          SELECT line_up, date, points
          FROM Matches
          WHERE (date BETWEEN '10/08/2019' AND '17/08/2019') AND points > 0
          """)
pd.DataFrame(c.fetchall(), columns=['line_up', 'date', 'points'])

,line_up,date,points
0,"['Salah', 'Milner', 'Henderson', 'Robertson', ...",14/12/2019,3
1,"['Alcantara', 'Van-Dijk', 'Jota', 'Robertson',...",15/09/2019,1
2,"['Alcantara', 'Becker', 'Robertson', 'Salah', ...",16/02/2020,1


#### 7. total cost for home match (at Anfield Stadium) for each supporter that payed more than 200£, and Liverpool won the game

In [641]:
c.execute("""
          SELECT l.supporter_id, m.rival_team, SUM(t.cost)
          FROM Matches m JOIN Tickets t on m.id = t.match_id JOIN LFCMembers l on t.supporter_id = l.supporter_id
          WHERE m.points = 3 AND m.location = 'Anfield'
          GROUP BY l.supporter_id
          HAVING SUM(t.cost) > 200
          """)
pd.DataFrame(c.fetchall(), columns=['supporter_id', 'rival_team', 'total_cost'])

,supporter_id,rival_team,total_cost
0,3628,Crystal Palace,369
1,5650,Crystal Palace,369
2,8750,Newcastle,294


# Bonus

#### IF/ELSE clause that determines for each player what is his paid status. Players that are getting paid more than 250 will be considered as 'highly paid' players, players that get more than 150 but less than or equal to 250 will be considered as 'reguarly paid' and players that are getting paid less than 150 will be considered as poorly paid

In [647]:
c.execute("""
          SELECT first_name, last_name,
          CASE
            WHEN salary > 250 THEN 'highly paid'
            WHEN salary > 150 AND salary <= 250 THEN 'paid regular'
            ELSE 'poorly paid'
          END AS paid_status
          FROM Players
          """)
pd.DataFrame(c.fetchall(), columns=['first_name', 'last_name', 'paid_status'])

,first_name,last_name,paid_status
0,Alisson,Becker,highly paid
1,Joel,Matip,paid regular
2,Joe,Gomez,paid regular
3,Ibrahima,Konate,paid regular
4,Virgil,Van-Dijk,highly paid
5,Trent,Alexander-Arnold,paid regular
6,Andrew,Robertson,paid regular
7,Jordan,Henderson,highly paid
8,James,Milner,poorly paid
9,Thiago,Alcantara,poorly paid


#### Using NTILE window function, which takes an integer value as an argument that divides the group into a number of integer values. So we divided the items at the LFC Shop to 5 groups by the amount of each item_id and sorted the table by the price.

In [657]:
c.execute("""
          SELECT item_id, amount, price, NTILE(5) OVER(ORDER BY amount) AS price_groups
          FROM LFCShop
          ORDER BY price
          """)

pd.DataFrame(c.fetchall(), columns=['item_id', 'amount', 'price', 'price_group'])

,item_id,amount,price,price_group
0,6213,114,220,4
1,8826,82,222,2
2,87,94,687,3
3,8814,31,795,1
4,2593,33,889,1
5,5705,123,964,4
6,6743,56,976,2
7,2596,155,1032,4
8,1121,148,1068,4
9,4174,173,1137,5
